In [1]:
# from src.dataset.process import process_data_pandas
# # data_file = "data/mini_sparkify_event_data.json"
# # data_file = "data/mini_sparkify_event_data_sample.json"
# data_file = "data/mini_sparkify_event_data.parquet/"
# gather_col = "userId"
# sort_col = "ts"
# alias = "data"
# target_col = "page"
# exploded_col = "explode"
# emb_out_col = "embeddings"
# target_out_col = "targets"
# process_data_pandas(
#     parquet_file=data_file,
#     gather_col=gather_col,
#     sort_col=sort_col,
#     target_col=target_col,
#     cache_size=1000*5,
#     device="cuda",
# )

In [ ]:
# import torch
# train_file = "data/train_dataset"
# test_file = "data/test_dataset"
# train = torch.load(train_file, map_location="cpu")
# tensors = train["tensors"]
# targets = train["targets"]
# length = len(tensors)
# midpoint = int(length/2)
# tensors1 = tensors[:midpoint]
# tensors2 = tensors[midpoint:]
# targets1 = targets[:midpoint]
# targets2 = targets[midpoint:]
# first_half_file = "data/train_first_half"
# second_half_file = "data/train_second_half"
# first_half = {
#     "tensors": tensors1,
#     "targets": targets1
# }
# second_half = {
#     "tensors": tensors2,
#     "targets": targets2
# }
# torch.save(first_half, first_half_file)
# torch.save(second_half, second_half_file)

In [ ]:
# first_half_file = "data/train_first_half"
# second_half_file = "data/train_second_half"
# import torch
# second_half = torch.load(second_half_file, map_location="cpu")
# tensors = second_half["tensors"]
# targets = second_half["targets"]
# length = len(tensors)
# midpoint = int(length/2)
# tensors1 = tensors[:midpoint]
# tensors2 = tensors[midpoint:]
# targets1 = targets[:midpoint]
# targets2 = targets[midpoint:]
# second_half1 = {
#     "tensors": tensors1,
#     "targets": targets1
# }
# second_half2 = {
#     "tensors": tensors2,
#     "targets": targets2
# }
# second_half1_file = "data/train_second_half1"
# second_half2_file = "data/train_second_half2"
# torch.save(second_half1, second_half1_file)
# torch.save(second_half2, second_half2_file)

In [ ]:
# import torch
# second_half1_file = "data/train_second_half1"
# second_half2_file = "data/train_second_half2"
# second_half2 = torch.load(second_half2_file, map_location="cpu")
# from datasets import Dataset
# ds = Dataset.from_dict(second_half2)
# ds.save_to_disk("data/train_second_half2_dataset")

In [ ]:
# files = [
#     "data/train_first_half_dataset",
#     "data/train_second_half1_dataset",
#     "data/train_second_half2_dataset"
# ]
# from datasets import load_from_disk
# ds = [
#     load_from_disk(file)
#     for file in files
# ]
# from datasets import concatenate_datasets
# combined = concatenate_datasets(ds)
# output = "train_dataset_combined"
# combined.save_to_disk(output)
# datasets = [load_from_disk(file) for file in files]

In [3]:
from src.dataset.dataset import SparkifyDataset

In [4]:
train_file = "data/train_dataset"

In [ ]:
ds = SparkifyDataset(train_file)

In [8]:
from src.preprocess.collate_fn import mat_collate_fn

In [9]:
from torch.utils.data import DataLoader

In [10]:
dl = DataLoader(ds, batch_size=2, shuffle=True, collate_fn=mat_collate_fn)

In [12]:
emb, pos_indices, targets, masks = next(iter(dl))

In [13]:
emb.shape

torch.Size([2, 128, 384])